# Calculate Phenotype Variant LD Values
Frank Grenn  
January 2020

### Input
* a `Variants.csv` file containing a `SNP`, `CHR`, `BP` and `Locus Number` column for each variant of interest (meta5 + progression)
* a `ranges.txt` file containing chr, bp position - 1MB, bp position + 1MB, and an id for each variant of interest. 
* a `Variant in other GWAS - Sheet1.csv` file containing the rsid, chr, bp, locus number, associated disease, amino acid change, and gene for each phenotype variant of interest
* plink binary files for the gwas data

### Output
* a `calculateLD.swarm` file that will use plink to get LD values for all the variants
* many `.log` files in the `out` directory generated by the notebook and swarm file containing the LD values
* a `PhenotypeVariantLD.csv` to the `results` directory containing all the phenotype variant's LD values in one file

In [ ]:
import pandas as pd
import subprocess
import re

In [18]:
variants = pd.read_csv("$PATH1/phenovars/LD/Variants.csv")
variants = variants.astype({'Locus Number':'str'})
variants

SNP  CHR         BP Locus Number
0   rs114138760    1  154898185            1
1    rs35749011    1  155135036            1
2    rs76763715    1  155205634            1
3     rs6658353    1  161469054            2
4    rs11578699    1  171719769            3
..          ...  ...        ...          ...
87   rs55818311   19    2341047           76
88   rs77351827   20    6006041           77
89    rs2248244   21   38852361           78
90   rs61863020   10  112956055        prog1
91     rs382940    9  108058562        prog2

[92 rows x 4 columns]

In [19]:
phenovars = pd.read_csv("$PATH1/phenovars/Variant in other GWAS - Sheet1.csv")
phenovars = phenovars.astype({'locus number':'str'})
print(len(phenovars.index))
print(phenovars.head())

375
          ID      location locus number  Chr     Start       End Ref Alt  \
0  rs3104413  543050:10:00           31    6  32582650  32582650   C   G   
1  rs3957146  544698:10:00           31    6  32681530  32681530   T   C   
2  rs3998159  544706:19:00           31    6  32682019  32682019   A   C   
3  rs6931277  543061:57:00           31    6  32583357  32583357   A   T   
4  rs7454108  544697:23:00           31    6  32681483  32681483   T   C   

  Func.refGene       Gene.refGene     GeneDetail.refGene ExonicFunc.refGene  \
0   intergenic  HLA-DRB1;HLA-DQA1  dist=25037;dist=22533                  .   
1   intergenic  HLA-DQB1;HLA-DQA2  dist=47064;dist=27633                  .   
2   intergenic  HLA-DQB1;HLA-DQA2  dist=47553;dist=27144                  .   
3   intergenic  HLA-DRB1;HLA-DQA1  dist=25744;dist=21826                  .   
4   intergenic  HLA-DQB1;HLA-DQA2  dist=47017;dist=27680                  .   

  AAChange.refGene                           other associated di

In [20]:
#drop identical rows
phenovars = phenovars.drop_duplicates()
print(len(phenovars.index))
print(phenovars.head())

370
          ID      location locus number  Chr     Start       End Ref Alt  \
0  rs3104413  543050:10:00           31    6  32582650  32582650   C   G   
1  rs3957146  544698:10:00           31    6  32681530  32681530   T   C   
2  rs3998159  544706:19:00           31    6  32682019  32682019   A   C   
3  rs6931277  543061:57:00           31    6  32583357  32583357   A   T   
4  rs7454108  544697:23:00           31    6  32681483  32681483   T   C   

  Func.refGene       Gene.refGene     GeneDetail.refGene ExonicFunc.refGene  \
0   intergenic  HLA-DRB1;HLA-DQA1  dist=25037;dist=22533                  .   
1   intergenic  HLA-DQB1;HLA-DQA2  dist=47064;dist=27633                  .   
2   intergenic  HLA-DQB1;HLA-DQA2  dist=47553;dist=27144                  .   
3   intergenic  HLA-DRB1;HLA-DQA1  dist=25744;dist=21826                  .   
4   intergenic  HLA-DQB1;HLA-DQA2  dist=47017;dist=27680                  .   

  AAChange.refGene                           other associated di

In [6]:
%%bash
mkdir $PATH1/phenovars/LD/out

In [21]:
for i in range(len(variants.index)):
    locus = variants.iloc[i]['Locus Number']
    #print(locus)
    
    locus_phenovars = phenovars.loc[phenovars['locus number'] == locus]
    #print(locus_phenovars)
    
    snp1 = str(variants.iloc[i]['CHR']) + ":" + str(variants.iloc[i]['BP'])

    if(len(locus_phenovars.index)!=0):
        for i in range(len(locus_phenovars.index)):

            snp2 = str(locus_phenovars.iloc[i]['Chr']) + ":" + str(locus_phenovars.iloc[i]['Start'])



            with open("$PATH1/phenovars/LD/calculateLD.swarm", 'a') as outfile:
                outfile.write("plink --bfile $PATH2/HARDCALLS_PD_september_2018_no_cousins --ld {} {} --extract range $PATH1/phenovars/LD/ranges.txt --out {}_{}\n".format(snp1,snp2,snp1,snp2))



In [ ]:
#run the swarm file
#cd $PATH1/phenovars/LD/out
#swarm -f $PATH1/phenovars/LD/calculateLD.swarm --partition quick --module plink

In [22]:
#read stuff
df = pd.DataFrame(columns=['rsid1','snp1','rsid2','snp2','r2','dprime'])
for i in range(len(variants.index)):
    locus = variants.iloc[i]['Locus Number']
    #print(locus)
    
    locus_phenovars = phenovars.loc[phenovars['locus number'] == locus]
    #print(locus_phenovars)
    
    snp1 = str(variants.iloc[i]['CHR']) + ":" + str(variants.iloc[i]['BP'])
    rsid1 = variants.iloc[i]['SNP']
    if(len(locus_phenovars.index)!=0):
        for i in range(len(locus_phenovars.index)):
            #reset the read string to null
            dataline='null'
            snp2 = str(locus_phenovars.iloc[i]['Chr']) + ":" + str(locus_phenovars.iloc[i]['Start'])
            rsid2 = locus_phenovars.iloc[i]['ID']
            #print("{} {}".format(snp1, snp2))


            
            file = open("$PATH1/phenovars/LD/out/"+str(snp1)+"_"+str(snp2)+".log","r")
            
            for line in file:
                if re.search("R-sq", line):
                    dataline = line
                    break
            
            #only add new data if 'R-sq' was found (meaning there was data in the log file and the 'null' value assigned earlier was overwritten)
            if(dataline!='null'):
                #mess with the strings
                dataline = dataline.strip('R-sq = ')
                dataline = dataline.strip(' ')
                splitdata = dataline.split("D' =")
                Rsq = splitdata[0]
                dprime = splitdata[1]
            
                df = df.append({'rsid1': rsid1,'snp1': snp1,'rsid2':rsid2, 'snp2': snp2, 'r2':Rsq.strip(' '), 'dprime':dprime.strip('\n')}, ignore_index = True)
        
print(len(df.index))
print(df.head())
print(df.tail())

447
         rsid1         snp1       rsid2         snp2           r2     dprime
0  rs114138760  1:154898185  rs12726330  1:155108167  0.000197778          1
1  rs114138760  1:154898185   rs2230288  1:155206167  0.000183319          1
2  rs114138760  1:154898185  rs35749011  1:155135036  0.000185332          1
3   rs35749011  1:155135036  rs12726330  1:155108167     0.929883   0.996158
4   rs35749011  1:155135036   rs2230288  1:155206167     0.937428   0.973508
         rsid1         snp1       rsid2         snp2        r2     dprime
442  rs8087969  18:48683589   rs8088085  18:48708548  0.977947   0.990706
443  rs8087969  18:48683589  rs16952939  18:48743642  0.923426   0.969977
444  rs8087969  18:48683589   rs1941961  18:48744150  0.672056   0.946768
445  rs8087969  18:48683589  rs62092949  18:48779673  0.718631   0.908091
446  rs8087969  18:48683589  rs57976196  18:48738645  0.921946   0.971478


In [23]:
df.to_csv("$PATH1/results/PhenotypeVariantLD.csv", index = False)